### Data augmentation process

In [ ]:
#!pip install rdkit

from augmentation import *

In [ ]:
test_smiles_list = ['CC1=C2C=C(C=C(C2=CC=C1)C(=O)O)[O-]']

In [ ]:
kekulize = get_kekulizeSmiles(test_smiles_list)
#output: ['CC1=C2C=C([O-])C=C(C(=O)O)C2=CC=C1']
hydrogen = get_ExplicitHs(test_smiles_list)
#output: ['CH3][c]1[cH][cH][cH][c]2[c]([C](=[O])[OH])[cH][c]([O-])[cH][c]12']
canonical = get_rdkitCanonical(test_smiles_list)
#output: ['Cc1cccc2c(C(=O)O)cc([O-])cc12']
cycle = get_cycleRenumering(test_smiles_list)
#output: ['CC1=C3C=C(C=C(C3=CC=C1)C(=O)O)[O-]']

### Model evaluation on augmented data

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("laituan245/molt5-large-smiles2caption", model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained('laituan245/molt5-large-smiles2caption')

input_text = 'CC1=C2C=C([O-])C=C(C(=O)O)C2=CC=C1'
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, num_beams=5, max_length=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

C:\Users\veron\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 1.21MB/s]
C:\Users\veron\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\veron\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Pytho

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("GT4SD/multitask-text-and-chemistry-t5-base-augm")
tokenizer = AutoTokenizer.from_pretrained("GT4SD/multitask-text-and-chemistry-t5-base-augm")

instance = "CC1=C2C=C([O-])C=C(C(=O)O)C2=CC=C1"
input_text = f"Caption the following molecule: {instance}"

text = tokenizer(input_text, return_tensors="pt")
output = model.generate(input_ids=text["input_ids"], max_length=max_length, num_beams=num_beams)
output = tokenizer.decode(output[0].cpu())

output = output.split(tokenizer.eos_token)[0]
output = output.replace(tokenizer.pad_token,"")
output = output.strip()

output